In [ ]:
import os
import numpy as np
import glob
import PIL.Image as Image
from tqdm.notebook import tqdm
import json
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import torchvision.models as models

We always check that we are running on a GPU

In [ ]:
if torch.cuda.is_available():
    print("The code will run on GPU.")
else:
    print("The code will run on CPU. Go to Edit->Notebook Settings and choose GPU as the hardware accelerator")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

We provide you with a class that can load the *hotdog/not hotdog* dataset you should use from /dtu/datasets1/02514/

In [ ]:
class Hotdog_NotHotdog(torch.utils.data.Dataset):
    def __init__(self, train, transform, data_path='/dtu/datasets1/02514/hotdog_nothotdog'):
        'Initialization'
        self.transform = transform
        data_path = os.path.join(data_path, 'train' if train else 'test')
        image_classes = [os.path.split(d)[1] for d in glob.glob(data_path +'/*') if os.path.isdir(d)]
        image_classes.sort()
        self.name_to_label = {c: id for id, c in enumerate(image_classes)}
        self.image_paths = glob.glob(data_path + '/*/*.jpg')
        
    def __len__(self):
        'Returns the total number of samples'
        return len(self.image_paths)

    def __getitem__(self, idx):
        'Generates one sample of data'
        image_path = self.image_paths[idx]
        
        image = Image.open(image_path)
        c = os.path.split(os.path.split(image_path)[0])[1]
        y = self.name_to_label[c]
        X = self.transform(image)
        return X, y

Below is the simple way of converting the images to something that can be fed through a network.
Feel free to use something other than $128\times128$ images.

In [ ]:
size = 128
jitter=0.5
train_transform1 = transforms.Compose([transforms.Resize((size, size)), 
                                      transforms.RandomRotation(degrees=(90)),
                                      transforms.ColorJitter(jitter),
                                    transforms.ToTensor()])
train_transform2 = transforms.Compose([transforms.Resize((size, size)), 
                                      transforms.RandomRotation(degrees=(90)),
                                      transforms.GaussianBlur(3),
                                    transforms.ToTensor()])
train_transform_orig = transforms.Compose([transforms.Resize((size, size)), 
                                    transforms.ToTensor()])

test_transform = transforms.Compose([transforms.Resize((size, size)), 
                                    transforms.RandomRotation(degrees=(90)),
                                      transforms.ColorJitter(jitter),
                                      transforms.GaussianBlur(3),
                                    transforms.ToTensor()])
batch_size = 64
trainset_trans1 = Hotdog_NotHotdog(train=True, transform=train_transform1)
trainset_trans2 = Hotdog_NotHotdog(train=True, transform=train_transform2)
trainset_orig = Hotdog_NotHotdog(train=True, transform=train_transform_orig)
all_train=trainset_trans1+trainset_trans2+trainset_orig

train_loader = DataLoader(all_train, batch_size=batch_size, shuffle=True, num_workers=3)
testset = Hotdog_NotHotdog(train=False, transform=test_transform)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=3)

Let's look at some images from our data 

In [ ]:
images, labels = next(iter(train_loader))
plt.figure(figsize=(20,10))

for i in range(21):
    plt.subplot(5,7,i+1)
    plt.imshow(np.swapaxes(np.swapaxes(images[i].numpy(), 0, 2), 0, 1))
    plt.title(['hotdog', 'not hotdog'][labels[i].item()])
    plt.axis('off')


Remember to save the state of your model AND optimizer regularly in case the Colab runtime times out.
You can save your model to your google drive, so you can get it from there in a new colab session. 

If you only save it in the colab notebook, there's no way to get it into a new session.

Now create a model and train it!


In [ ]:
#from ResNet import ResNet152
#resnet=ResNet152(2).to('cuda')

First we are going to use a very basic neural network

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class BasicNetwork(nn.Module):
    def __init__(self, n_filters, out_features=2):
        super().__init__()
        self.layers = nn.Sequential(
                        nn.Conv2d(3, n_filters, kernel_size=4, stride=2, padding=1), 
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.ReLU(),
                        nn.Conv2d(n_filters, n_filters*2, kernel_size=4, stride=2, padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.ReLU()
        )
        self.linear = nn.Linear(n_filters*2*8*8, 2)
    def forward(self, x):
        out = self.layers(x)
        out = self.linear(out.view(x.size(0), -1))
        return out

In [ ]:
class BasicNetwork_2(nn.Module):
    """ BasicNetwork + Batchnorm """
    def __init__(self, n_filters, out_features=2):
        super().__init__()
        self.layers = nn.Sequential(
                        nn.Conv2d(3, n_filters, kernel_size=4, stride=2, padding=1), 
                        nn.BatchNorm2d(n_filters),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.BatchNorm2d(n_filters),
                        nn.ReLU(),
                        nn.Conv2d(n_filters, n_filters*2, kernel_size=4, stride=2, padding=1),
                        nn.BatchNorm2d(n_filters*2),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.BatchNorm2d(n_filters*2),
                        nn.ReLU()
        )
        self.linear = nn.Linear(n_filters*2*8*8, 2)
        
    def forward(self, x):
        out = self.layers(x)
        out = self.linear(out.view(x.size(0), -1))
        return out

In [ ]:
class BasicNetwork_3(nn.Module):
    """ BasicNetwork + Batchnorm + Dropout """
    def __init__(self, n_filters, out_features=2):
        super().__init__()
        self.layers = nn.Sequential(
                        nn.Conv2d(3, n_filters, kernel_size=4, stride=2, padding=1), 
                        nn.BatchNorm2d(n_filters),
                        nn.Dropout(0.3),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.BatchNorm2d(n_filters),
                        nn.Dropout(0.3),
                        nn.ReLU(),
                        nn.Conv2d(n_filters, n_filters*2, kernel_size=4, stride=2, padding=1),
                        nn.BatchNorm2d(n_filters*2),
                        nn.Dropout(0.3),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2),
                        nn.BatchNorm2d(n_filters*2),
                        nn.Dropout(0.3),
                        nn.ReLU()
        )
        self.linear = nn.Linear(n_filters*2*8*8, 2)
        
    def forward(self, x):
        out = self.layers(x)
        out = self.linear(out.view(x.size(0), -1))
        return out

In [ ]:
#We define the training as a function so we can easily re-use it.
def train(model, optimizer, loss_fun, name, num_epochs=30, from_checkpoint=False):
    
    if from_checkpoint:
        model.load_state_dict(torch.load(os.path.join("checkpoints",name)))
    
    out_dict = {'train_acc': [],
              'test_acc': [],
              'train_loss': [],
              'test_loss': [],
              'predictions': [],
              'target': []}
  
    for epoch in tqdm(range(num_epochs), unit='epoch'):
        model.train()
        #For each epoch
        train_correct = 0
        train_loss = []
        model.train()
        for minibatch_no, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
            data, target = data.to(device), target.to(device)
            #Zero the gradients computed for each weight
            optimizer.zero_grad()
            #Forward pass your image through the network
            output = model(data)
            #Compute the loss
            loss = loss_fun(output, target)
            
            #Backward pass through the network
            loss.backward()
            #Update the weights
            optimizer.step()

            train_loss.append(loss.item())
            #Compute how many were correctly classified
            predicted = output.argmax(1)
            train_correct += (target==predicted).sum().cpu().item()
        #Comput the test accuracy
        test_loss = []
        test_correct = 0
        model.eval()
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            with torch.no_grad():
                output = model(data)
            test_loss.append(loss_fun(output, target).cpu().item())
            predicted = output.argmax(1)
            test_correct += (target==predicted).sum().cpu().item()
            out_dict["predictions"].append(predicted)
            out_dict["target"].append(target)
        out_dict['train_acc'].append(train_correct/len(all_train))
        out_dict['test_acc'].append(test_correct/len(testset))
        out_dict['train_loss'].append(np.mean(train_loss))
        out_dict['test_loss'].append(np.mean(test_loss))
        print(f"Loss train: {np.mean(train_loss):.3f}\t test: {np.mean(test_loss):.3f}\t",
              f"Accuracy train: {out_dict['train_acc'][-1]*100:.1f}%\t test: {out_dict['test_acc'][-1]*100:.1f}%")
        
        # Saving weights
        torch.save(model.state_dict(), os.path.join("checkpoints",name))
        
        # Saving training/testing info
        info_df = pd.DataFrame()
        info_df['train_acc'] = np.array(out_dict['train_acc']).reshape(-1)
        info_df['test_acc'] = np.array(out_dict['test_acc']).reshape(-1)
        info_df['train_loss'] = np.array(out_dict['train_loss']).reshape(-1)
        info_df['test_loss'] = np.array(out_dict['test_loss']).reshape(-1)
        info_df.to_csv(name+"_info.csv", index=False)
        
        
    return out_dict

In [ ]:
# Our networks
basicnetwork = BasicNetwork(4).to(device)
basicnetwork2 = BasicNetwork_2(4).to(device)
basicnetwork3 = BasicNetwork_3(4).to(device)

# Some state of the art networks
resnet152 = models.resnet152(num_classes=2, pretrained=False).to(device)
alexnet = models.alexnet(num_classes=2, pretrained=False).to(device)
vgg16 = models.vgg16(num_classes=2, pretrained=False).to(device)

# CrossE ntropy Loss function
loss_fun = nn.CrossEntropyLoss()

In [ ]:
## SGD and adam optimizers
optimizer_SGD = optim.SGD(basicnetwork.parameters(), lr=0.01)
optimizer_Adam = optim.Adam(basicnetwork.parameters(), lr=0.001)

In [ ]:
out_dict = train(resnet152, optimizer_SGD, loss_fun, "ResNet152_SGD", num_epochs=50)
out_dict = train(alexnet, optimizer_SGD, loss_fun, "AlexNet_SGD", num_epochs=50)
out_dict = train(vgg16, optimizer_SGD, loss_fun, "VGG16_SGD", num_epochs=50)

out_dict = train(resnet152, optimizer_Adam, loss_fun, "ResNet152_Adam", num_epochs=50)
out_dict = train(alexnet, optimizer_Adam, loss_fun, "AlexNet_Adam", num_epochs=50)
out_dict = train(vgg16, optimizer_Adam, loss_fun, "VGG16_Adam", num_epochs=50)

out_dict = train(basicnetwork, optimizer_SGD, loss_fun, "BasicNetwork_SGD", num_epochs=50)
out_dict = train(basicnetwork2, optimizer_SGD, loss_fun, "BasicNetwork_2_SGD", num_epochs=50)
out_dict = train(basicnetwork3, optimizer_SGD, loss_fun, "BasicNetwork_3_SGD", num_epochs=50)

out_dict = train(basicnetwork, optimizer_Adam, loss_fun, "BasicNetwork_Adam", num_epochs=50)
out_dict = train(basicnetwork2, optimizer_Adam, loss_fun, "BasicNetwork_2_Adam", num_epochs=50)
out_dict = train(basicnetwork3, optimizer_Adam, loss_fun, "BasicNetwork_3_Adam", num_epochs=50)